<a href="https://colab.research.google.com/github/averrows/ml/blob/main/TubesB_13520007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Besar IF3270 Pembelajaran Mesin Semester 2 Tahun 2022/2023**
### **Bagian A: Implementasi Forward Propagation untuk Feed Forward Neural Network**
Disusun oleh:
*   Nadia Mareta Putri Leiden - 13520007 - K1
*   Taufan Fajarama Putrawansyah R - 13520031 - K1
*   Raden Haryosatyo Wisjnunandono - 13520070 - K2
*   Averrous Saloom - 13520100 - K2








---


**Nomor 1.**
Rancanglah struktur file teks eksternal yang akan dipakai untuk menyimpan model dalam bentuk struktur jaringan (jumlah layer, fungsi aktivasi dan jumlah neuron per layer) dan bobot dari FFNN. Setiap layer dibatasi memiliki neuron dengan fungsi aktivasi yang sama, sedangkan antar layer diperbolehkan memiliki neuron dgn fungsi aktivasi yang berbeda. Fungsi aktivasi yang dikenali adalah linear, sigmoid, ReLU, dan softmax.
*   linear: f(net)=net
*   reLU: f(net)=max{0,net}
*   sigmoid: f(net)=1/(1+e^(-net))
*   softmax: f(net_i)=e^(net_i)/Σe^(net_i)

Berikut struktur/format file teks eksternal yang akan dipakai untuk menyimpan model


```
jumlah_layer
fungsi_aktivasi_1,jumlah_neuron_1
fungsi_aktivasi_2,jumlah_neuron_2
...
fungsi_aktivasi_n,jumlah_neuron_n
bias_layer_1
bobot_layer_1
bias_layer_2
bobot_layer_2
...
bias_layer_n
bobot_layer_n
```



Berikut contoh isi file teks eksternal (model1.txt)


```
2
sigmoid,2
relu,2
0.2,0.3
0.0,-2.0,-1.0,0.0
0.4,0.5
0.0,-3.0,-1.0,0.0
```





---


**Nomor 2.**
Implementasi fitur load (membaca) model FFNN dari file teks.

In [ ]:
import numpy as np


def load_model(filename):
  with open(filename, 'r') as f:
    lines = f.read().split('\n')

  num_layer = int(lines[0])
  network_info = []
  biases = []
  weights = []
  counter = 0
  layer = 0

  # membaca line file teks
  for i, line in enumerate(lines):
    if i > 0:
      # membaca informasi jaringan (fungsi aktivasi & jumlah neuron per layer)
      if i <= num_layer:
        layer_info = line.strip().split(',')
        network_info.append((layer_info[0], int(layer_info[1])))
      else:
        # membaca bias per layer
        if counter == 0:
          bias = [float(x) for x in lines[i].strip().split(',')]
          biases.append(np.array(bias))
          counter += 1
        # membaca weight per layer
        elif counter == 1:
          weight = []
          neurons = lines[i].strip().split(',')
          neurons = np.array_split(neurons, len(neurons)/network_info[layer-1][1])
          for j in range(len(neurons)):
            row = [float(x) for x in neurons[j]]
            weight.append(row)
          weights.append(np.array(weight))
          counter = 0
          layer += 1
  
  # menyusun model jaringan
  model = []
  for i, layer in enumerate(network_info):
    activation_function = layer[0]
    num_neurons = layer[1]
    model.append({
      'activation_function': activation_function,
      'num_neurons': num_neurons,
      'biases': biases[i],
      'weights': weights[i]
    })
    
  return model

In [ ]:
# fungsi input
def input_to_model(arr, model):
  num_neurons_input = len(arr[0])
  model.insert(0, {
    'num_neurons': num_neurons_input,
    'value': arr
  })
  return model

# test load & input model
model = load_model('model1.txt')
val_input = np.array([[1.0, 0.0], [0.0, 1.0], [0.0, 0.0]])
model = input_to_model(val_input, model)
print(model)

[{'num_neurons': 2, 'value': array([[1., 0.],
       [0., 1.],
       [0., 0.]])}, {'activation_function': 'sigmoid', 'num_neurons': 2, 'biases': array([0.2, 0.3]), 'weights': array([[ 0., -2.],
       [-1.,  0.]])}, {'activation_function': 'relu', 'num_neurons': 2, 'biases': array([0.4, 0.5]), 'weights': array([[ 0., -3.],
       [-1.,  0.]])}]




---


**Nomor 3.a.**
Implementasi forward propagation untuk FFNN dengan kemampuan:
*   Menampilkan model berupa struktur jaringan dan bobotnya, formatnya bebas.

In [ ]:
# bikin fungsi visualisasi jaringan
from graphviz import Digraph
import random


def drawModel(model, filename) :
    f = Digraph('G', filename='./graph_{}'.format(filename))
    f.attr('node', shape='circle')

    num_input = 0

    for layer in range (1, len(model)):
        weight = np.transpose(model[layer]["weights"])
        for i in range (model[layer]["num_neurons"]):
            for j in range (model[layer-1]["num_neurons"]):
                if (layer == 1):
                    start = "x" + str(j+1)
                else:
                    start = "h" + str(layer-1) + "," + str (j+1)
                if(layer == (len(model)-1)) :
                    end = "y" + str(i+1)              
                else:
                    end = "h" + str(layer) + "," + str (i+1)
                f.edge(start,end,label=str(weight[i][j]))

            name_bias = "b" + str(layer)
            if (layer == (len(model)-1)):
                end = "y" + str(i+1)
            f.edge(name_bias,end,label=str(model[layer]["biases"][i]))

    f.view()
    print("==========================================")
    print(f)

    print("==========================================")
    for layer in range (1, len(model)):
        print("LAYER       {}: {}".format(layer,model[layer]["activation_function"]))   

In [ ]:
# implement
drawModel(model, "model1")

digraph G {
	node [shape=circle]
	x1 -> "h1,1" [label=0.0]
	x2 -> "h1,1" [label=-1.0]
	b1 -> "h1,1" [label=0.2]
	x1 -> "h1,2" [label=-2.0]
	x2 -> "h1,2" [label=0.0]
	b1 -> "h1,2" [label=0.3]
	"h1,1" -> y1 [label=0.0]
	"h1,2" -> y1 [label=-1.0]
	b2 -> y1 [label=0.4]
	"h1,1" -> y2 [label=-3.0]
	"h1,2" -> y2 [label=0.0]
	b2 -> y2 [label=0.5]
}

LAYER       1: sigmoid
LAYER       2: relu


**Nomor 3.b.**
Implementasi forward propagation untuk FFNN dengan kemampuan:
*   Memprediksi output untuk input 1 instance.

In [ ]:
# fungsi aktivasi
def sigmoid(value: np.ndarray) -> np.ndarray:     
  return 1 / (1 + np.exp(-value))

def relu(value: np.ndarray) -> np.ndarray:
  return np.maximum(0, value)

def linear(value: np.ndarray) -> np.ndarray:
  return value

def softmax(value: np.ndarray) -> np.ndarray:
  e_i = np.exp(value - np.max(value))
  return e_i / e_i.sum()

functions = {
    "sigmoid": sigmoid,
    "relu": relu,
    "linear": linear,
    "softmax": softmax
}

# fungsi FFNN
def forward_propagation(model):
  results = []
  for inputs in range(len(model[0]["value"])):
    output_val = model[0]["value"][inputs]
    for layer in range (1, len(model)):
      output_val = feed_forward(layer, model, output_val)
    results.append(output_val)
  return results

def feed_forward(layer, model, prevoutput):
  output_val = np.dot(prevoutput, model[layer]["weights"]) + model[layer]['biases']
  a_fun = functions[model[layer]["activation_function"]] 
  return a_fun(output_val)

In [ ]:
# implementasi 1 instance
model = load_model('model1.txt')
val_input = np.array([[1.0, 0.0]])
model = input_to_model(val_input, model)
print(model)
print()
res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[1., 0.]])}, {'activation_function': 'sigmoid', 'num_neurons': 2, 'biases': array([0.2, 0.3]), 'weights': array([[ 0., -2.],
       [-1.,  0.]])}, {'activation_function': 'relu', 'num_neurons': 2, 'biases': array([0.4, 0.5]), 'weights': array([[ 0., -3.],
       [-1.,  0.]])}]

[array([0.24553473, 0.        ])]


**Nomor 3.c.**
Implementasi forward propagation untuk FFNN dengan kemampuan:
*   Memprediksi output untuk input batch sejumlah instances.

In [ ]:
# implementasi sejumlah instances
model = load_model('model1.txt')
val_input = np.array([[1.0, 0.0], [0.0, 1.0], [0.0, 0.0]])
model = input_to_model(val_input, model)
print(model)
print()
res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[1., 0.],
       [0., 1.],
       [0., 0.]])}, {'activation_function': 'sigmoid', 'num_neurons': 2, 'biases': array([0.2, 0.3]), 'weights': array([[ 0., -2.],
       [-1.,  0.]])}, {'activation_function': 'relu', 'num_neurons': 2, 'biases': array([0.4, 0.5]), 'weights': array([[ 0., -3.],
       [-1.,  0.]])}]

[array([0.24553473, 0.        ]), array([0., 0.]), array([0., 0.])]




---


**Nomor 4.** Pengujian kebenaran fungsional dari forward propagation yang diimplementasikan dilakukan dengan menggunakan kasus uji yang disiapkan oleh asisten.

In [ ]:
# linear.json
model = load_model('linear_asisten.txt')
val_input = np.array([[3.0,1.0]])
model = input_to_model(val_input, model)
print(model)
print()

drawModel(model, 'linear_asisten')
print()

res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[3., 1.]])}, {'activation_function': 'linear', 'num_neurons': 3, 'biases': array([0.2, 0.3, 0.1]), 'weights': array([[ 0.5,  0.2, -0.8],
       [ 0.3, -0.6,  0.4]])}]

digraph G {
	node [shape=circle]
	x1 -> y1 [label=0.5]
	x2 -> y1 [label=0.3]
	b1 -> y1 [label=0.2]
	x1 -> y2 [label=0.2]
	x2 -> y2 [label=-0.6]
	b1 -> y2 [label=0.3]
	x1 -> y3 [label=-0.8]
	x2 -> y3 [label=0.4]
	b1 -> y3 [label=0.1]
}

LAYER       1: linear

[array([ 2. ,  0.3, -1.9])]


In [ ]:
# relu.json
model = load_model('relu_asisten.txt')
val_input = np.array([[-1.0, 0.5]])
model = input_to_model(val_input, model)
print(model)
print()

drawModel(model, 'relu_asisten')
print()

res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[-1. ,  0.5]])}, {'activation_function': 'relu', 'num_neurons': 3, 'biases': array([0.1, 0.2, 0.3]), 'weights': array([[ 0.4, -0.5,  0.6],
       [ 0.7,  0.8, -0.9]])}]

digraph G {
	node [shape=circle]
	x1 -> y1 [label=0.4]
	x2 -> y1 [label=0.7]
	b1 -> y1 [label=0.1]
	x1 -> y2 [label=-0.5]
	x2 -> y2 [label=0.8]
	b1 -> y2 [label=0.2]
	x1 -> y3 [label=0.6]
	x2 -> y3 [label=-0.9]
	b1 -> y3 [label=0.3]
}

LAYER       1: relu

[array([0.05, 1.1 , 0.  ])]


In [ ]:
# sigmoid.json
model = load_model('sigmoid_asisten.txt')
val_input = np.array([[0.2, 0.4]])
model = input_to_model(val_input, model)
print(model)
print()

drawModel(model, 'sigmoid_asisten')
print()

res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[0.2, 0.4]])}, {'activation_function': 'sigmoid', 'num_neurons': 3, 'biases': array([0.4, 0.2, 0.1]), 'weights': array([[0.2, 0.4, 0.2],
       [0.1, 0.2, 0.4]])}]

digraph G {
	node [shape=circle]
	x1 -> y1 [label=0.2]
	x2 -> y1 [label=0.1]
	b1 -> y1 [label=0.4]
	x1 -> y2 [label=0.4]
	x2 -> y2 [label=0.2]
	b1 -> y2 [label=0.2]
	x1 -> y3 [label=0.2]
	x2 -> y3 [label=0.4]
	b1 -> y3 [label=0.1]
}

LAYER       1: sigmoid

[array([0.61774787, 0.58904043, 0.57444252])]


In [ ]:
# softmax.json
model = load_model('softmax_asisten.txt')
val_input = np.array([[1.0, 2.0]])
model = input_to_model(val_input, model)
print(model)
print()

drawModel(model, 'softmax_asisten')
print()

res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[1., 2.]])}, {'activation_function': 'softmax', 'num_neurons': 3, 'biases': array([1., 2., 3.]), 'weights': array([[2., 1., 3.],
       [3., 2., 1.]])}]

digraph G {
	node [shape=circle]
	x1 -> y1 [label=2.0]
	x2 -> y1 [label=3.0]
	b1 -> y1 [label=1.0]
	x1 -> y2 [label=1.0]
	x2 -> y2 [label=2.0]
	b1 -> y2 [label=2.0]
	x1 -> y3 [label=3.0]
	x2 -> y3 [label=1.0]
	b1 -> y3 [label=3.0]
}

LAYER       1: softmax

[array([0.66524096, 0.09003057, 0.24472847])]


In [ ]:
# multilayer.json
model = load_model('multilayer_asisten.txt')
val_input = np.array([[1.0, 0.0],[0.0, 1.0],[0.0, 0.0]])
model = input_to_model(val_input, model)
print(model)
print()

drawModel(model, 'multilayer_asisten')
print()

res = forward_propagation(model)
print(res)

[{'num_neurons': 2, 'value': array([[1., 0.],
       [0., 1.],
       [0., 0.]])}, {'activation_function': 'linear', 'num_neurons': 2, 'biases': array([0.5, 0.5]), 'weights': array([[ 0., -2.],
       [-1.,  0.]])}, {'activation_function': 'relu', 'num_neurons': 2, 'biases': array([0.5, 0.5]), 'weights': array([[ 0., -3.],
       [-1.,  0.]])}]

digraph G {
	node [shape=circle]
	x1 -> "h1,1" [label=0.0]
	x2 -> "h1,1" [label=-1.0]
	b1 -> "h1,1" [label=0.5]
	x1 -> "h1,2" [label=-2.0]
	x2 -> "h1,2" [label=0.0]
	b1 -> "h1,2" [label=0.5]
	"h1,1" -> y1 [label=0.0]
	"h1,2" -> y1 [label=-1.0]
	b2 -> y1 [label=0.5]
	"h1,1" -> y2 [label=-3.0]
	"h1,2" -> y2 [label=0.0]
	b2 -> y2 [label=0.5]
}

LAYER       1: linear
LAYER       2: relu

[array([2., 0.]), array([0., 2.]), array([0., 0.])]


**Perbandingan Nilai**

In [ ]:
import json

def load_test_case(filename):
  f = open(filename, 'r')
  content = json.load(f)

  test_case = content["case"]

  model = build_case_model(test_case["model"]["layers"], test_case["weights"])
  input = test_case["input"]

  # put input into model
  model = input_to_model(input, model)

  expected_output = content["expect"]["output"]
  maximum_error = content["expect"]["max_sse"]
  return model, expected_output, maximum_error
  
def build_layer(activation_function, num_neurons, bias, weight):
  return {
      'activation_function': activation_function,
      'num_neurons': num_neurons,
      'biases': bias,
      'weights': weight
  }

def build_case_model(layers, weights):
  model = []
  for idx, layer in enumerate(layers):
    model.append(build_layer(layer["activation_function"], 
                             layer["number_of_neurons"], 
                             np.array(weights[idx][0]),
                             np.array(weights[idx][1:])))
  return model

def sse(A,B):
  dif = A.ravel() - B.ravel()
  return np.dot( dif, dif )

def run_test_case(filename):
  model, expected, maximum_error = load_test_case(filename)
  output = forward_propagation(model)

  print("TESTCASE: ", filename)
  print("EXPECTED")
  print(expected)
  print("RESULT")
  print(list(map(lambda x: x.tolist(),output)))
  print("ERROR")
  err = sse(np.array(expected), np.array(output))
  print(err, " Maximum: ", maximum_error)
  if (err <= maximum_error):
    print("========================= PASS ==========================")
  else:
    print("========================= FAIL ==========================")

def run_test():
  test_cases = ['linear.json','multilayer.json','relu.json','sigmoid.json','softmax.json']
  for test in test_cases:
    run_test_case(test)

run_test()

TESTCASE:  linear.json
EXPECTED
[[2.0, 0.3, -1.9]]
RESULT
[[2.0, 0.3000000000000001, -1.9000000000000004]]
ERROR
2.0954117794933126e-31  Maximum:  1e-06
========================= PASS ==========================
TESTCASE:  multilayer.json
EXPECTED
[[2.0, 0.0], [0.0, 2.0], [0.0, 0.0]]
RESULT
[[2.0, 0.0], [0.0, 2.0], [0.0, 0.0]]
ERROR
0.0  Maximum:  1e-06
========================= PASS ==========================
TESTCASE:  relu.json
EXPECTED
[[0.05, 1.1, 0.0]]
RESULT
[[0.04999999999999996, 1.1, 0.0]]
ERROR
1.7333369499485123e-33  Maximum:  1e-06
========================= PASS ==========================
TESTCASE:  sigmoid.json
EXPECTED
[[0.617747], [0.58904], [0.574442]]
RESULT
[[0.617747874769249, 0.5890404340586651, 0.574442516811659]]
ERROR
1.2207224545374987e-12  Maximum:  1e-06
========================= PASS ==========================
TESTCASE:  softmax.json
EXPECTED
[[0.665241, 0.090031, 0.244728]]
RESULT
[[0.6652409557748218, 0.09003057317038046, 0.24472847105479764]]
ERROR
4.060320